In [1]:
# Packages used
import pygbif as gbif
import pyinaturalist as pyna
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import Polygon

INFO:NumExpr defaulting to 4 threads.


In [24]:
# functions
def obtain_inat (taxon_id_r, nelat=0, nelng=0, swlat=0, swlng=0, bounding_box=True):
    
    nuevo_final=pd.DataFrame()
    
    if bounding_box==True:
        a= pyna.v1.observations.get_observations(taxon_id=taxon_id_r, 
                                                 page=1, per_page=200, 
                                                 geo=True,
                                                 nelat=nelat,nelng=neln,
                                                 swlat=swlat,swlng=swlng)
    else:
        a= pyna.v1.observations.get_observations(taxon_id=924150, page=1, per_page=200, geo=True,
                                                 quality_grade='research', verifiable=True)
    si=int(a['total_results']/a['per_page']) ## como solo imprime por pagina 200 se itera hasta conseguir todas las paginas disponibles
    for e in range(si+1):
            if bounding_box==True:
                a= pyna.v1.observations.get_observations(taxon_id=taxon_id_r,
                                                         page=e+1, per_page=200,
                                                         geo=True,
                                                         nelat=nelat,nelng=neln,
                                                         swlat=swlat,swlng=swlng)
            else:
                a= pyna.v1.observations.get_observations(taxon_id=taxon_id_r, page=1, per_page=200, geo=True,
                                                         quality_grade='research', verifiable=True)
            nuevo=pd.DataFrame()
            for elemento in a['results']:
                name=elemento['taxon']['name']
                latitud=elemento['location'][0]
                longitud=elemento['location'][1]
                place=elemento['place_guess']
                final=pd.DataFrame({"name":[name], "latitud": [latitud],
                                   "longitud": [longitud], "place":[place]})
                nuevo=pd.concat([nuevo, final])
            nuevo_final=pd.concat([nuevo,nuevo_final])
    nuevo_final.columns = ['species', 'decimalLatitude', 'decimalLongitude', 'stateProvince']
    return nuevo_final

In [13]:
def obtain_gbif (especie):
    ocurrences=gbif.occurrences.search(scientificName=especie, limit= 300,hasCoordinate=True, basisOfRecord=["MATERIAL_CITATION",
                                                                                                             "PRESERVED_SPECIMEN"])
    ##### se necesita solo cambiar el genus key para que pueda ser leido de manera mejorada en la base de datos
    x=pd.DataFrame()
    for datum in ocurrences['results']:
        diccionario={ e: [datum[e]] for e in datum.keys() }
        data=pd.DataFrame(diccionario)
        try:
            data=data[['genus','species', 'decimalLatitude','decimalLongitude', 'country', 'stateProvince', 'occurrenceID']]
        except KeyError:
            pass
            #data=data[['genus', 'decimalLatitude','decimalLongitude', 'country', 'occurrenceID']]
        x=pd.concat([x, data])
    return x

# List of species Gbif and Inaturalist

In [3]:
taxon_id_r=61267 ### este es el taxon id de heteroptera
nelat=-17.5
nelng=-55.5
swlat=-34.5
swlng=-68.5

In [ ]:
nuevo_final= obtain_inat (taxon_id_r,nelat=nelat,nelng=nelng,swlat=swlat,swlng=swlng, bounding_box=True)
lista=[e for e in nuevo_final["species"].unique() if " " in e] # this is the list of species in inaturalist

# Download ocurrences of each species

In [46]:
# para sacar ocurrencias de gbif

In [22]:
especie="Panstrongylus megistus"
species_id=924150

In [14]:
x= obtain_gbif(especie)
nuevo_final=obtain_inat(species_id, bounding_box=False)
x["Autor"]="gbif"
nuevo_final["Autor"]="Inaturalist"

In [15]:
if len(x)>0: # solo cuando se recupera alguna especie con ese nombre
    if "stateProvince" in x.keys():
        x=x[['genus', 'species', 'decimalLatitude','decimalLongitude',"stateProvince","country","Autor"]]
    if "stateProvince" not in x.keys():
        x=x[['genus', 'species', 'decimalLatitude','decimalLongitude',"country","Autor"]] # muchas veces en gbif no hay state province

In [33]:
final=pd.concat([x,nuevo_final])

In [34]:
# Rename to could be used in posterior analysis
final = final.rename(columns={'species':'Especie',
                              'decimalLatitude':'Latitud',
                              "decimalLongitude":"Longitud",
                              "stateProvince":"Localidad",
                                "country":"Pais"})

In [35]:
if len(final)>0: # solo cuando se recupera alguna especie con ese nombre
    if "Pais" in final.keys():
        final=final[['genus', 'Especie', 'Latitud',"Longitud","Localidad","Pais","Autor"]]
    if "Pais" not in x.keys():
        final=final[[ 'Especie', 'Latitud',"Longitud","Localidad","Autor"]]

In [36]:
# add the year of search of the information and drop duplicates to make easier to see
final["Año"]=2024
final = final.drop_duplicates()

In [87]:
final.to_excel(paste(especie, ".xlsx", sep="")